In [ ]:
from huggingface_hub import notebook_login
hf_token=userdata.get('hftoken')
from transformers import AutoTokenizer,AutoModelForCausalLM
model_name="google/gemma-2b"
model=AutoModelForCausalLM.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)
print(model)


In [ ]:
input_text="What should I do if i face an issue"
input_ids=tokenizer(input_text,return_tensor="pt")
outputs=model.generate(**input_ids,max_length=120)
print(tokenizer.decode(outputs[0]))


In [ ]:
## Based on Databricks dolly

In [ ]:
## Step2: Motivation for PEFT

In [ ]:
from datasets import load_dataset
dataset_name="databricks/databricks-dolly-15k"
dataset=load_dataset(dataset_name,split="train[0:1000]")


In [ ]:
def formatting_prompts(example):
    output_texts=[]
    for i in range(len(example['instruction'])):
        if example['category'][i] in ['open_qa','general_qa']:
            text=f"Instruction:\n{example['instruction']}\n\nResponse:\n{example['response']}"
            output_texts.append(text)
    return output_texts

trainer=SFTTrainer(model,train_dataset=dataset,tokenizer=tokenizer,formatting_func=formatting_prompts)
print("Intialized trainer for training")
trainer.train()

In [ ]:
## Finetuning using Parameter Effecient FineTuning

In [ ]:
## Create own dataset

In [ ]:
from datasets import load_dataset
dataset_name="databricks/databricks-dolly-15k"
dataset=load_dataset(dataset_name,split="train")


In [ ]:
from collections import defaultdict
filtered_dataset=[]
categories,count=defaultdict(int)
for _,data in enumerate(dataset):
    if data["context"]:
        continue
    else:
        text=f"Instruction:\n{data['instruction']}\n\nResponse:\n{data['response']}"
        filtered_dataset.append(text)
print(filtered_dataset[0:2])


In [ ]:
import jsonlines as jl
with jl.open('dolly-mini-train.jsonl','w') as writers:
    writer.write_all(filtered_dataset[0:])

In [ ]:
from datasets import load_dataset
dataset_name="ai-bites/databricks-mini"
dataset=load_dataset(dataset_name,split="train[0:100]")
dataset

In [ ]:
model_name="google/gemma-2b"
new_model="gemma-ft"
lora_r=64
lora_alpha=16
lora_dropout=0.1
use_4bit=True
## Quantization type3
## Two stges of quaninzation 1. Standard and Double Quantization
bnb_4bit_quant_type="nf4"
output_dir='./results'
num_train_epochs=1
fp16=False
bf16=False
per_device_train_batch_size=2
gradient_accumulation_steps=1
gradient_checkpointing=True
max_grad_norm=0.3
learning_rate="2e-4"
weight_decay=0.001
optim='paged_adam3_32bit'
lr_scheduler_type="constant"
max_steps=-1
warmup_ration=0.03
group_by_length=True
save_steps=25
logging_steps=25
max_seq_length=128
packing=True
device_map={"":0}
device_map="auto"

In [ ]:
## Load QLORA Configuration

In [ ]:
compute_dtype=getattr(torch,bnb_4bit_compute_dtype)
bnb_config=BitsAndBytesConfig(load_in_4bit,bnb_4bit_quant_type=bnb_4bit_quant_type,bnb_4bit_compute_dtype=compute_dtype,bnb_4bit_use_double_quant=use_nested_quant,)


In [ ]:
## Checking GPU Comptability

In [ ]:
if compute_dtype==torch.float16 and use_4bit:
    major,_=torch.cuda.get_device_capability()
    if major>=8:
        print("Setting BF16 to True")
        bf16=True
    else:
        bs16=False

In [ ]:
## Load the model and Tensor

In [ ]:
## Loading basemodel
model=AutoModelForCausalLM.from_pretrained(model)

In [ ]:
training_arguments=TrainingArguments(output_dir=output_dir,num_train_epochs=num_train_epochs,report_to="tensorboard",learning_rate=learning_rate,max_grad_norm=max_grad_norm,max_steps=max_steps,warmup_ratio=warmup_ratio,fp16=fp16,group_by_length=group_by_length)

In [ ]:
trainer=SFTTrainer(model=model,train_dataset=dataset,peft_config=peft_config,packing=packing,max_seq_length=max_seq_length,tokenizer=tokenizer,formatting_func=formatting_prompts)


In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
## Visualize Training on tensorboard

In [ ]:
## Incase of overfitting we can do two things Augugmentation, addiing more data to the training dataset

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs


In [ ]:
## Prompting the newly fine tuned model

In [ ]:
input_text="What should I do on a trip to Europe?"
base_model=AutoModelForCausalLM.from_pretrained(model_name,low_cpu_mem_usage=True,return_dict=True,torch_dtype=torch.float16,device_map=device_map)
model=PeftModel.from_pretrained(base_model,new_model)
model=model.merge()

## Reload tokenizer to save it
tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [ ]:
## second Model

In [ ]:
import os
os.environ
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AuotTokenizer,AutoConfig,AutoModelForCasualLM
model=AutoModelForCasualLM.from_pretrained("bigscience/bloom-7bl",load_in_8bit=True,device_map='auto')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("bigscience/bloom-7bl")

In [ ]:
## Freezing the weights

In [ ]:
for param in model.parameters:

In [ ]:
from peft import LoraConfig,get_peft_model
config=LoraConfig(r=16,lora_alpha=32,lora_dropout=0.06,bias="none",task_type="CASUAL_LM")
model=get_peft_model(model,config)
print_trainable_parameters(model)

In [ ]:
## Data
import transformers
from datasets import load_dataset
data=load_dataset("Abirate/english_quotes")


In [ ]:
def merge_columns(example):
    example["prediction"]=example["quote"]+"->:"+   str(example["tags"])
    return example

data['train']=data['train'].map(merge_columns)
data['train']['prediction'][:5]


In [ ]:
data=data.map(lambda samples:tokenizer(samples['prediction']),batched=True)

In [ ]:
## Training
train=transformers.Trainer(model=model,train_dataset=data['train'],args=Transformers.TrainingArguments(per_device_train_batch_size=4,gradient_accumulation_steps=4,warmup_steps=100,max_steps=200,learning_rate=2e-4,fp16=True,logging_steps=1,output_dir='outputs'),data_collator=transformers.DataCollatroForLanguageModeling(tokenizer,mlm=False))
model.config.use_cache=False
trainer.train()

In [ ]:
## sharing the adapters on the hub

In [ ]:
model.push_to_hub("samwit/bloom-7bl-lora-tagger",use_auth_token=True,commit_message="basic training",private=True)

In [ ]:
## Load adapaters from Hub

In [ ]:
import torch
from peft import PeftModel,PeftConfig
from transformers import AutoModelForCasualLM,AutoTokenizer
peft_model_id="samwit/bloom-7b-lora-tagger"
config=PeftConfig.from_pretrained(peft_model_id)
model=AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,return_dict=True,load_in_8bit=True)
Tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model=PeftModel.from_pretrained(model,peft_model_id)

In [ ]:
batch=  tokenizer("the world is your carrer ->",return_tensors="pt")
with torch.cuda.amp.autocast():
    output_tokens=model.generate(**batch,max_new_tokens=50)
print('\n \n',tokenizer.decode(output_tokens[0],skip_special_token=True))

In [ ]:
# https://www.youtube.com/watch?v=Us5ZFp16PaU&t=518s